In [55]:
# Dependencies
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colormaps as cmaps
from matplotlib.colors import ListedColormap as lcmap
from matplotlib.colors import Normalize as norm
from matplotlib.colors import LogNorm as logn
import matplotlib.animation as animation
import fns as f

In [59]:
ca_bd = gpd.read_file('../outputs/geojson/ca_boundary.geojson')
ca_bbox = gpd.read_file('../outputs/geojson/ca_bbox.geojson')

In [70]:
# Get dates to pull fire data
dt = [f'{str(x.date())[0:7]}' for x in pd.date_range("2012-01-01", "2024-02-29", freq="ME")]
dt[13:]

['2013-02',
 '2013-03',
 '2013-04',
 '2013-05',
 '2013-06',
 '2013-07',
 '2013-08',
 '2013-09',
 '2013-10',
 '2013-11',
 '2013-12',
 '2014-01',
 '2014-02',
 '2014-03',
 '2014-04',
 '2014-05',
 '2014-06',
 '2014-07',
 '2014-08',
 '2014-09',
 '2014-10',
 '2014-11',
 '2014-12',
 '2015-01',
 '2015-02',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',
 '20

In [6]:
ca_fire_gdf = gpd.read_file('../outputs/nasa-firms/geojson/ca_fire_cells.geojson')

In [9]:
ca_fire_gdf

,geometry
0,"POLYGON ((-122.10000 38.00000, -122.10000 38.1..."
1,"POLYGON ((-118.30000 34.20000, -118.30000 34.3..."
2,"POLYGON ((-118.50000 34.30000, -118.50000 34.4..."
3,"POLYGON ((-120.40000 37.70000, -120.40000 37.8..."
4,"POLYGON ((-122.40000 37.90000, -122.40000 38.0..."
...,...
3233,"POLYGON ((-120.00000 34.80000, -120.00000 34.9..."
3234,"POLYGON ((-120.10000 34.80000, -120.10000 34.9..."
3235,"POLYGON ((-119.90000 34.70000, -119.90000 34.8..."
3236,"POLYGON ((-120.40000 39.20000, -120.40000 39.3..."


In [51]:
rain_gdf_dict = {}
for d in dt:
    df = pd.read_csv(f'../outputs/nasa-gesdisc/{d}.csv').set_index('lon')
    rain = []
    for i in ca_fire_gdf.index:
        x = round(ca_fire_gdf.loc[i,'geometry'].centroid.x,2)
        y = round(ca_fire_gdf.loc[i,'geometry'].centroid.y,2)
        rain.append(df.loc[x,str(y)])
    rain_gdf_dict[d] = gpd.GeoDataFrame({'rain':rain},geometry=ca_fire_gdf.geometry,crs=4326)

In [ ]:
rain_gdf_dict[d]

In [56]:
# set custom colomap
afmhot = cmaps['viridis']
viridis2 = lcmap(afmhot(np.linspace(0.3, 0.8, 256)))

In [60]:
# Create Plots
for d in dt:
    for c in ['rain']: #,'cnt','frp/cnt']
        # Plot ca cells
        ax = rain_gdf_dict[d].plot(figsize=(11,10), column=c, cmap=viridis2, alpha=0.8,
                                 markersize=1, vmin=0, vmax=300, zorder=3)
        ax.axis('off')
        ca = ca_bd.to_crs(crs=4326).plot(ax=ax, color='#333', zorder=2) # Plot state boundary
        bbox = ca_bbox.to_crs(crs=4326).plot(ax=ax, color='#fff', zorder=1) # Plot bounding box
        plt.colorbar(cm.ScalarMappable(cmap=viridis2), ax=ax, 
                     location='right', anchor=(0, 0.3), shrink=0.5) # Configure colorbar
        v = rain_gdf_dict[d][c]
        plt.title(f'{d}: min={v.min():.{2}f} median={np.median(v):.{1}f} max={v.max():.{0}f}', loc='left')
        plt.savefig(f'../outputs/nasa-gesdisc/{c}_{d}.png')
        plt.close()

In [61]:
df1 = pd.DataFrame(columns=dt)
for d in dt:
    df1[d] = rain_gdf_dict[d]['rain']

In [65]:
boxprops = dict(linestyle='-', linewidth=0.5, color='#999', facecolor="#eee")
flierprops = dict(marker='o', markerfacecolor='#999', markersize=3, markeredgecolor='none', alpha=0.4)
medianprops = dict(linestyle='-', linewidth=2.5, color='green')
meanprops = dict(marker='D', markeredgecolor='none',markerfacecolor='#ffa425')
capprops = dict(color="#666", linewidth = 2.5)

In [76]:
df1[12:].T.plot.bar(ylabel='precipitation mm/mo', 
            figsize=(55,27), grid=False, stacked=True)
plt.title('Total Monthly Precipitation in 0.1° Grid')
plt.xticks(rotation='vertical')
plt.grid(axis='y',color='#eee')
# plt.yscale('log')
# plt.savefig(f'../outputs/nasa-gesdisc/boxplot_log.png')
plt.show()

ValueError: Image size of 4365x67665 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 5500x2700 with 1 Axes>